In [ ]:
import rasterio
from rasterio.features import geometry_mask
from rasterio.fill import fillnodata
import geopandas as gpd
import numpy as np

import fiona
from fiona.crs import from_epsg

from rasterio.features import shapes
from shapely.geometry import shape, mapping

from scipy.ndimage import generic_filter


### Create a nodata mask

In [ ]:
# Read the DEM and nanify every no data cell
dem = rasterio.open('C:/Users/PaulBillecocq/Documents/UdS/KRG_DATA/a3d-prep/dem/DEM-hrdps-grid-extent-5m.tif')
dem_array = dem.read(1)
# dem_array[dem_array < 0] = np.nan

In [ ]:
# Create a nodata mask
nodata_mask = np.isnan(dem_array)

### Identify nan areas (lakes and sea) to apply the right elevation interpolation method 

In [ ]:
# Create a geopackage holding polygons for every no data shape (e.g. lakes and sea)
results = (
    {'properties': {'raster_val': v}, 'geometry': s}
    for i, (s, v) in enumerate(shapes(dem_array, mask=nodata_mask, transform=dem.transform))
)

schema = {
    'geometry': 'Polygon',
    'properties': {'raster_val': 'int'},
}

with fiona.open('./nodata_polygons.gpkg', 'w', 'GPKG', schema, crs=from_epsg(2951)) as gpkg:
    for result in results:
        gpkg.write(result)

# Lakes and sea are identified manually using QGIS at this point

### Elevation interpolation
Sea pixels should be set to 0, lake pixels should be interpolated according to their neighborhood

In [ ]:
# Load polygons
gdf = gpd.read_file('./nodata_polygons.gpkg')

# Create masks for sea and lake with buffer for lake masks
sea_mask = geometry_mask([geom for geom in gdf[gdf['Type'] == 'sea'].geometry], transform=dem.transform, invert=True, out_shape=dem.shape)
#lake_mask = geometry_mask([geom.buffer(1) for geom in gdf[gdf['Type'] == 'lake'].geometry], transform=dem.transform, invert=True, out_shape=dem.shape)

# Set sea areas to 0
dem_array[sea_mask] = 0

In [ ]:
lake_mask[lake_mask == True] = 0

In [ ]:
dem.nodata

In [ ]:
nodata_value = dem.nodata if dem.nodata is not None else -9999
dem_array = fillnodata(image=dem_array, mask=(dem_array == nodata_value), max_search_distance=1)

In [ ]:
# Save the modified DEM
profile = dem.profile
profile.update(
    dtype=dem_array.dtype,
    driver='GTiff'
)
with rasterio.open('C:/Users/PaulBillecocq/Documents/UdS/KRG_DATA/a3d-prep/dem/DEM-hrdps-grid-extent-5m-cleaned.tif', 'w', **profile) as dst:
    dst.write(dem_array, 1)

In [ ]:
dem_array.shape